## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открываем файл с данными и изучаем общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
#Общие сведения о данных
df.info()
#Проверка названий столбцов
df.columns
#первые несколько строк датафрейма
df.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод:  из 21525 строк только 19351 не имеют пропусков. Строки с пропуском в столбце трудового стажа так же имеют пропуск в столбце ежемесячного дохода, что не может быть случайностью.

## Шаг 2. Предобработка данных

## 2.1 Обработка пропусков

- Пропуски в столбце дохода в месяц можно заполнить медианными значениями для каждого типа занятости.

In [2]:
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


- Исследуем столбец 'days_employed'.

In [3]:
#проверим на наличие отрицательных значений
#print(df[df['days_employed'] < 0])
#проверим на наличие связи отрицательного стажа с другими переменными
print(df[df['days_employed'] < 0].head(20))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   
13         0   -1846.641941         54  неоконченное высшее             2   

- Зависимости отрицательного стажа от других переменных не обнаружено. Так как в задании не оговаривается трудовой стаж, а так же по причине сложности заполнения пропусков и работы с отрицательными значениями, его можно исключить из дальнейшего анализа.

In [4]:
df = df.drop('days_employed',1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод: получена таблица без пропусков. Причиной пропусков может быть банальный отказ клиента указывать данные о доходе и трудовом стаже, что в свою очередь приводит к бОльшему "проценту" по кредиту.

## 2.2 Замена типа данных

- Замена типа данных необходима только для столбца ежемесячного дохода. (тип float64 на int64)

In [5]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Вывод: получена таблица без пропусков и типами данных, необходимых для дальнейшего анализа.

## 2.3 Обработка дубликатов

- Перед поиском дубликатов привожу текстовые значения в нижний регистр

In [6]:
df['education'] = df['education'].str.lower()
df['purpose'] = df['purpose'].str.lower()
df.duplicated().sum()

71

- Удаление 71 строк (менее 0,5% от общего числа строк) никак не повлияет на дальнейший анализ.

In [7]:
df = df.drop_duplicates().reset_index(drop=True)

In [8]:
#проверка удаления дубликатов
df.duplicated().sum()

0

### Вывод: получена таблица без пропусков, дубликатов и типами данных, необходимых для дальнейшего анализа.

## 2.4 Лемматизация

In [9]:
#импорт библиотеки
from pymystem3 import Mystem
m = Mystem()
#сразу добавляем столбец с леммами, объедененными в строку
df['purpose_lemma'] = df['purpose'].apply(lambda x: ''.join(m.lemmatize(x)))

In [10]:
#вычисляем самые частоиспользуемые леммы
from collections import Counter
print(Counter(df['purpose_lemma']))

Counter({'автомобиль\n': 972, 'свадьба\n': 791, 'на проведение свадьба\n': 768, 'сыграть свадьба\n': 765, 'операция с недвижимость\n': 675, 'покупка коммерческий недвижимость\n': 661, 'операция с жилье\n': 652, 'покупка жилье для сдача\n': 651, 'операция с коммерческий недвижимость\n': 650, 'покупка жилье\n': 646, 'жилье\n': 646, 'покупка жилье для семья\n': 638, 'строительство собственный недвижимость\n': 635, 'недвижимость\n': 633, 'операция со свой недвижимость\n': 627, 'строительство жилой недвижимость\n': 624, 'покупка недвижимость\n': 621, 'покупка свой жилье\n': 620, 'строительство недвижимость\n': 619, 'ремонт жилье\n': 607, 'покупка жилой недвижимость\n': 606, 'на покупка свой автомобиль\n': 505, 'заниматься высокий образование\n': 496, 'сделка с подержанный автомобиль\n': 486, 'на покупка подержать автомобиль\n': 478, 'свой автомобиль\n': 478, 'на покупка автомобиль\n': 471, 'приобретение автомобиль\n': 461, 'дополнительный образование\n': 460, 'сделка с автомобиль\n': 455, '

In [11]:
#склеиваем все строки стобца целей кредита
purposes = ' '.join(df['purpose'])
#проводим лемматизацию
purposes_lem = m.lemmatize(purposes)
#считаем количесвто вхождений каждой отдельной леммы
print(Counter(purposes_lem))


Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


### Вывод: получив количество вхождений целей и отдельных лемм можно с легкостью выделить основные группы в целях кредита.`

In [13]:
#группируем данные по леммам и добавляем столбец в датафрейм
def group(row):
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'свадьба' in row:
        return 'свадьба'
    if 'образование' in row:
        return 'образование'
    if 'недвижимость' in row:
        return 'недвижимость'
    if 'жилье' in row:
        return 'недвижимость'

    
df['group_lemma'] = df['purpose_lemma'].apply(group)   

### Вывод: получена таблица с категоризацией по леммам, без пропусков, дубликатов, и типами данных, необходимых для дальнейшего анализа.

## 2.5 Категоризация данных

In [14]:
#удаляем строки с отрицательным значением в столбце 'children'
df = df[df['children'] != -1]

In [15]:
#Категоризация по доходу, границы групп - квантили
df['group_income'] = pd.qcut(df['total_income'],4)
#Категоризация на молодежь,средний возраст, средний возраст+достаточно взрослые дети, пенсионеры
df['group_years'] = pd.cut(df['dob_years'], [18,25,45,65,999])
#Категоризация по количеству детей
def group(x):
    if x == 0:
        return '0 детей'
    if x>0 and x<=2:
        return '1-2 детей'
    if x >= 3:
        return '>3 детей'
df['group_child'] = df['children'].apply(group)

In [16]:
#вывод конечной таблицы
df

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,group_lemma,group_income,group_years,group_child
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n,недвижимость,"(195868.5, 2265604.0]","(25, 45]",1-2 детей
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n,автомобиль,"(107555.0, 142594.0]","(25, 45]",1-2 детей
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n,недвижимость,"(142594.0, 195868.5]","(25, 45]",0 детей
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n,образование,"(195868.5, 2265604.0]","(25, 45]",>3 детей
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n,свадьба,"(142594.0, 195868.5]","(45, 65]",0 детей
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,операция с жилье\n,недвижимость,"(195868.5, 2265604.0]","(25, 45]",1-2 детей
21450,0,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,сделка с автомобиль\n,автомобиль,"(142594.0, 195868.5]","(65, 999]",0 детей
21451,1,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,недвижимость\n,недвижимость,"(20666.999, 107555.0]","(25, 45]",1-2 детей
21452,3,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,на покупка свой автомобиль\n,автомобиль,"(195868.5, 2265604.0]","(25, 45]",>3 детей


### Вывод: получена таблица без пропусков, дубликатов, и типами данных, необходимых для дальнейшего анализа. А так же добавлены столбы с категориями.

In [17]:
#создаем словарь для столбца образование
edu_dict = df[['education','education_id']]
edu_dict.drop_duplicates().reset_index(drop=True)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [18]:
#создаем словарь для столбца образование
edu_dict = df[['family_status','family_status_id']]
edu_dict.drop_duplicates().reset_index(drop=True)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


## Шаг 3. Ответы на вопросы

Далее в таблицах приведены количество всех клиентов, количество и доля должников.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
df.pivot_table(index = 'group_child',values = 'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
group_child,,,
0 детей,14091,1063,0.075438
1-2 детей,6860,638,0.093003
>3 детей,456,39,0.085526


### Вывод: категория с наименьшим риском невозврата кредита это клиенты без детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
df.pivot_table(index = 'family_status',values = 'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2805,274,0.097683
в разводе,1191,85,0.071369
вдовец / вдова,955,63,0.065969
гражданский брак,4146,388,0.093584
женат / замужем,12310,930,0.075548


### Вывод: категории не женат/не замужем и гражданский брак имеют бОльшую долю должников.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
df.pivot_table(index = 'group_income',values = 'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
group_income,,,
"(20666.999, 107555.0]",5352,427,0.079783
"(107555.0, 142594.0]",5468,483,0.088332
"(142594.0, 195868.5]",5235,447,0.085387
"(195868.5, 2265604.0]",5352,383,0.071562


### Вывод: все категории имеют 7-9% должников. Уровень дохода слабо влияет на вероятность возврата кредита в срок, однако видно, что клиенты с самым низким и самым высоким доходом более надежны, чем "средний класс".

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
df.pivot_table(index = 'group_lemma',values = 'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
group_lemma,,,
автомобиль,4295,402,0.093597
недвижимость,10787,782,0.072495
образование,4003,370,0.092431
свадьба,2322,186,0.080103


### Вывод: наименьшее количество просрочек по кредиту в категории недвижимость.

## Шаг 4. Общий вывод

- Нами были обработаны входные данные: удалены дубликаты, заполнены пропуски, проведена леммизация целей кредита и категоризация
- Выявлено, что категории с большим риском невозврата кредита в срок это клиенты, имеющие 1-2 детей, а также клиенты находящиеся в гражданском браке и не женатые/не замужем.
- В задании не говорилось, но так же приведена таблица с заисимостью возврата кредита в срок от образования клиента, из которой можем сделать вывод, что клиенты с начальным образованием являются наиболее рискованной категорией.

In [23]:
df.pivot_table(index = 'education',values = 'debt', aggfunc = ['count','sum','mean'])

,count,sum,mean
,debt,debt,debt
education,,,
высшее,5241,278,0.053043
начальное,282,31,0.109929
неоконченное высшее,743,68,0.091521
среднее,15135,1363,0.090056
ученая степень,6,0,0.000000
